# [LAB 4] Multivariable linear regression

- [강의 동영상 4-1](https://youtu.be/fZUV3xjoZSM)
- [강의 동영상 4-2](https://youtu.be/o2q4QNnoShY)
- [강의 슬라이드 4-1, 4-2](https://docs.google.com/presentation/d/1WF5yphSXyzYLG8wmVvOpRmgAlw4vewbK51ZwLAOFZXk)


# [LAB 4-1] Multivariable linear regression

### Hypothesis using matrix

$$H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3$$
![test-scores-for-general-psychology](images/lab4/test-scores-for-general-psychology.png)

- 퀴즈점수1(x1), 퀴즈점수2(x2), 퀴즈점수3(x3) 인 경우 기말시험(Y)에서 몇점이 나올지 예측하는 모델
- x가 1, 2, 3, ... 으로 증가할수록 `prediction power`가 더 세진다.
- 이 실습에서는 는 학습해야 할 weight이 3개
- b(bias)는 수식 간략화를 위해 일단 생략

### 실습코드 1 (원시적인 코드)

#### 수식표현 이용
- $H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3$

In [15]:
import tensorflow as tf

x1_data = [73., 93., 89., 96., 73.]
x2_data = [80., 88., 91., 98., 66.]
x3_data = [75., 93., 90., 100., 70.]
y_data = [152., 185., 180., 196., 142.]

# Placeholders for a tensor that will be always fed.
x1 = tf.placeholder(tf.float32)
x2 = tf.placeholder(tf.float32)
x3 = tf.placeholder(tf.float32)

Y = tf.placeholder(tf.float32)

w1 = tf.Variable(tf.random_normal([1]), name='weight1')
w2 = tf.Variable(tf.random_normal([1]), name='weight2')
w3 = tf.Variable(tf.random_normal([1]), name='weight3')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = (x1 * w1) + (x2 * w2) + (x3 * w3) + b

# Cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize. Need a very small learning rate for this data set
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session
sess = tf.Session()
# Initializes global variables in the graph
sess.run(tf.global_variables_initializer())
for step in range(2001):
    cost_val, hypothesis_val, _ = sess.run([cost, hypothesis, train],
                                  feed_dict={x1: x1_data, x2: x2_data, x3: x3_data, Y: y_data})
    if step % 100 == 0:
        print(step, "Cost: ", cost_val, ", Prediction(hypothesis):", hypothesis_val)

0 Cost:  4965.869 , Prediction(hypothesis): [ 91.86105  107.72876  107.780014 114.48096   83.85599 ]
100 Cost:  2.8868432 , Prediction(hypothesis): [154.31282 182.89737 181.7893  195.08601 141.20815]
200 Cost:  2.7821145 , Prediction(hypothesis): [154.25778 182.93468 181.77188 195.07768 141.25354]
300 Cost:  2.6827214 , Prediction(hypothesis): [154.20413 182.97101 181.75491 195.06964 141.29764]
400 Cost:  2.5884073 , Prediction(hypothesis): [154.1519  183.00638 181.73836 195.0619  141.34052]
500 Cost:  2.498839 , Prediction(hypothesis): [154.10101 183.04088 181.72221 195.05446 141.38217]
600 Cost:  2.4138412 , Prediction(hypothesis): [154.0514  183.07442 181.70647 195.04727 141.42264]
700 Cost:  2.3331676 , Prediction(hypothesis): [154.00313 183.10709 181.69112 195.04037 141.462  ]
800 Cost:  2.256532 , Prediction(hypothesis): [153.95605 183.13892 181.67615 195.03375 141.50023]
900 Cost:  2.1837454 , Prediction(hypothesis): [153.91022 183.16995 181.66156 195.0274  141.5374 ]
1000 Cost:

### 결과 분석
- step 증가할수록 cost 낮아짐
- step 증가할수록 prediction(hypothesis)가 y_data에 수렴하게 된다.
- 따라서 정확도 증가시키기 위해서는 step 증가시키면 된다.

### 개선사항
- x1, x2, ... 갯수가 많아지는 경우 이와같은 코드는 보기도 어렵고, 작성시에도 더 지저분한 등 문제가 많음
- 다음 실습에서 Matrix를 이용하여 이 코드를 다시 작성해본다.

### 실습코드 2 (실습1 코드 개선 버전)
#### Matrix 표현 이용
- $H(X) = XW$
- $\begin{pmatrix} x_1 \ x_2 \ x_3\end{pmatrix}\cdot\begin{pmatrix}w_1 \\ w_2 \\ w_3\end{pmatrix} = \begin{pmatrix}x_1w_1 + x_2w_2 + x_3w_3\end{pmatrix}$

In [6]:
import tensorflow as tf

# 실습1 코드 버전
# x1_data = [73., 93., 89., 96., 73.]
# x2_data = [80., 88., 91., 98., 66.]
# x3_data = [75., 93., 90., 100., 70.]
# y_data = [152., 185., 180., 196., 142.]

# 실습2 코드 버전
x_data = [[73.,80.,75.],[93.,88.,93.],[89.,91.,90.],[96.,98.,100.],[73.,66.,70.]]
y_data = [[152.], [185.], [180.], [196.], [142.]]

# Placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3]) # [x data의 row 갯수(미정=가변=n개인 경우 none으로 지정) , x의 column 갯수=3]
Y = tf.placeholder(tf.float32, shape=[None, 1]) # [y data의 row 갯수(미정=가변=n개인 경우 none으로 지정) , w의 column 갯수=1]

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b # matrix multiplication
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hypothesis_val, _ = sess.run([cost, hypothesis, train],
                                  feed_dict={X: x_data, Y: y_data})
    if step % 100 == 0:
        print(step, "\nCost: ", cost_val, "\nPrediction(hypothesis):\n", hypothesis_val, "\n")

0 
Cost:  60982.918 
Prediction(hypothesis):
 [[-69.45417 ]
 [-77.609955]
 [-79.46488 ]
 [-85.53145 ]
 [-58.815075]] 

100 
Cost:  1.3941259 
Prediction(hypothesis):
 [[149.691  ]
 [185.7277 ]
 [180.03696]
 [197.05206]
 [142.03691]] 

200 
Cost:  1.3616947 
Prediction(hypothesis):
 [[149.72137]
 [185.70724]
 [180.04675]
 [197.05536]
 [142.01328]] 

300 
Cost:  1.3308265 
Prediction(hypothesis):
 [[149.75099]
 [185.68732]
 [180.0563 ]
 [197.0585 ]
 [141.99033]] 

400 
Cost:  1.3014162 
Prediction(hypothesis):
 [[149.77988]
 [185.66791]
 [180.06564]
 [197.06146]
 [141.96805]] 

500 
Cost:  1.2734472 
Prediction(hypothesis):
 [[149.808  ]
 [185.649  ]
 [180.07474]
 [197.06429]
 [141.9464 ]] 

600 
Cost:  1.2467906 
Prediction(hypothesis):
 [[149.83543]
 [185.63057]
 [180.08363]
 [197.06697]
 [141.92538]] 

700 
Cost:  1.2213917 
Prediction(hypothesis):
 [[149.86218]
 [185.61263]
 [180.0923 ]
 [197.0695 ]
 [141.90498]] 

800 
Cost:  1.1971982 
Prediction(hypothesis):
 [[149.88826]
 [185.59

### 결과 분석
- step 증가할수록 cost 낮아짐
- step 증가할수록 prediction(hypothesis)가 y_data에 수렴하게 된다.
- 따라서 정확도 증가시키기 위해서는 step 증가시키면 된다.
- **(당연히)** 모든 결과가 실험코드1과 같고, 코드는 간단해졌다.
- Matrix를 사용하면 variable의 갯수가 몇개이든 상관없이 쉽게 표현하고 구현할 수 있다! (앞으로 계속 이렇게 할 것임!)

# [LAB 4-2] Loading Data from File

- data가 많은 경우, data를 소스코드에 넣어주는 것보다 파일로부터 불러오는 것이 유리하다. 이를 위한 방법을 알아보자!

### Hypothesis using matrix

$$H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3$$
![test-scores-for-general-psychology](images/lab4/test-scores-for-general-psychology.png)

- 퀴즈점수1(x1), 퀴즈점수2(x2), 퀴즈점수3(x3) 인 경우 기말시험(Y)에서 몇점이 나올지 예측하는 모델
- x가 1, 2, 3, ... 으로 증가할수록 `prediction power`가 더 세진다.
- 이 실습에서는 는 학습해야 할 weight이 3개
- b(bias)는 수식 간략화를 위해 일단 생략

### (참고)  Indexing, Slicing, Iterating
- 행렬 계산시에 자주 사용하는 메소드

In [72]:
# Slicing in Python

# range(a,b) = a는 포함, b는 미포함 즉, a <= range(a,b) < b
nums = range(5) # range(0,5) : means "[0, 1, 2, 3, 4]"
nums[2:4] # range(2,4)
nums[2:] # range(2,5)
nums[:2] # range(0,2)
nums[:] # range(0,5) 전체 다
nums[:-1] # range(0,5) 마지막 인덱스 제외

range(0, 4)

In [84]:
# Slicing in Numpy
import numpy as np

arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
# arr

#idx 0  1  2  3
# ----------------
# [[ 1  2  3  4]
#  [ 5  6  7  8]
#  [ 9 10 11 12]]
# ----------------

# arr[row, col]
arr[:,0] # [1 5 9]

# 인자 하나만 입력시 semicolon(:) 또는 dots(...) 입력된 것과 결과 같음
arr[-1] # [ 9 10 11 12]
arr[-1, :] # [ 9 10 11 12] 바로 위와 같음
arr[-1, ...] # [ 9 10 11 12] 바로 위와 같음 (':' = '...')

arr[0:2, :]
# [[1 2 3 4]
#  [5 6 7 8]]
arr[0:2, 0:2]
# [[1 2]
#  [5 6]]

[[1 2]
 [5 6]]


### (참고) 파일에서 데이터 로드한 후, 로드가 제대로 잘 되었는지 확인하는 방법

In [100]:
import numpy as np

xy = np.loadtxt('data/data-01-test-score.csv', delimiter=',', dtype=np.float32)
# print(xy)
# [[ 73.  80.  75. 152.]
#  [ 93.  88.  93. 185.]
#  [ 89.  91.  90. 180.]
# ...
x_data = xy[:,0:-1]
y_data = xy[:, [-1]]

# print(x_data) # x_data 전부다 출력됨
# print(y_data) # y_data 전부다 출력됨

# Make sure the shape and data are OK
print(x_data.shape) # (25,3) 몇열 몇행인지
print(len(x_data)) # 25 행 수
print(x_data) # ... 데이터


(25, 3)
25
[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]]


### 실습코드

In [33]:
# import data/data-01-test-score.csv
# CSV 파일은 ","로 데이터들이 구분되어 있음

# === file example ===
# x1,x2,x3,y

# 73,80,75,152
# 93,88,93,185
# 89,91,90,180
# 96,98,100,196
# ...

import numpy as np

xy = np.loadtxt('data/data-01-test-score.csv', delimiter=',', dtype=np.float32)
# print(xy)
# [[ 73.  80.  75. 152.]
#  [ 93.  88.  93. 185.]
#  [ 89.  91.  90. 180.]
# ...

# xy[row, col]

x_data = xy[:,0:-1] # 마지막 인덱스 제외 (즉, 처음부터 마지막에서 두번째까지)
y_data = xy[:,[-1]]

# Make sure the shape and data are OK
# print(x_data, x_data.shape, len(x_data))
# print(y_data.shape, y_data)

# Placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))
# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())
# Set up feed_dict variables inside the loop.
for step in range(2001):
    cost_val, hypothesis_val, _ = sess.run([cost, hypothesis, train], feed_dict={X:x_data, Y:y_data})
    if step % 100 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hypothesis_val)

[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]]
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]]
0 Cost:  101169.57 
Prediction:
 [[-147.5893  ]
 [-169.939   ]
 [-171.14767 ]
 [-188.81264 ]
 [-125.616684]
 [ -95.19896 ]
 [-146.81042 ]
 [-116.77909 ]
 [-159.90489 ]
 [-152.60657 ]
 [-136.73291 ]
 [-132.37274 ]
 [-174.27594 ]
 [-140.85953 ]
 [-146.61725 ]
 [-175.41364 ]
 [-128.89171 ]
 [-179.5715  ]
 [-166.90796 ]
 [-150.52438 ]
 [-170.34253 ]


In [132]:
# Ask score (Prediction!)
print("\n* Your score will be \n", sess.run(hypothesis, feed_dict={X:[[100, 70, 101]]}))
print("\n* Friends' scores will be \n", sess.run(hypothesis, feed_dict={X:[[100, 70, 101],[60, 70, 110],[90, 100, 80]]}))


* Your score will be 
 [[192.96066]]

* Friends' scores will be 
 [[192.96066]
 [143.41489]
 [183.68434]]


### Queue Runners
- numpy를 써서 로드하기에 파일이 너무 커서 메모리에 한번에 올리기도 어려운 경우에 사용할 수 있음
- https://www.tensorflow.org/programmers_guide/reading_data

#### 작동 순서
- 여러 개의 파일을 읽어온다고 할 때
- 여러 개의 파일을 큐에 쌓고
- Reader로 읽고
- 데이터 양식에 맞게 decoder에서 decode (콤마를 제거한다든지...)
- 읽어온 결과를 다시 큐에 쌓는다.
- 학습시킬 때는 batch만큼 읽어서 학습을 시킨다.


### Queue Runners 실습코드
- https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-04-4-tf_reader_linear_regression.py 참조

In [1]:
import tensorflow as tf

# 3가지 step으로 나누어볼 수 있음

# 1st step: file들의 list를 만들어 준다.
filename_queue = tf.train.string_input_producer(['data/data-01-test-score.csv'], shuffle=False, name='filename_queue')

# 2nd step: file을 읽어올 reader 정의
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# 3rd step: 어떤 형식으로 읽어올지 decode 정의
record_defaults = [[0.],[0.],[0.],[0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# tf.train.batch로 읽기
# collect batches of csv in
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

# placeholders for a tensor that will be always fed
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X,W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hypothesis_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hypothesis_val)

coord.request_stop()
coord.join(threads)

# # Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

0 Cost:  2636.4446 
Prediction:
 [[ 91.02282 ]
 [129.05148 ]
 [116.970634]
 [129.81163 ]
 [101.000854]
 [ 79.8847  ]
 [ 94.7818  ]
 [ 66.490234]
 [127.80534 ]
 [125.07114 ]]
10 Cost:  93.06326 
Prediction:
 [[144.01823 ]
 [192.69289 ]
 [179.70592 ]
 [198.12952 ]
 [149.52895 ]
 [115.647514]
 [145.92705 ]
 [104.41745 ]
 [187.26335 ]
 [180.12651 ]]
20 Cost:  95.45637 
Prediction:
 [[144.40804]
 [193.09724]
 [180.13805]
 [198.59248]
 [149.8277 ]
 [115.84851]
 [146.28023]
 [104.69049]
 [187.61708]
 [180.43153]]
30 Cost:  94.8457 
Prediction:
 [[144.44272 ]
 [193.07498 ]
 [180.14969 ]
 [198.59753 ]
 [149.80109 ]
 [115.80968 ]
 [146.29068 ]
 [104.709404]
 [187.57211 ]
 [180.36739 ]]
40 Cost:  94.218506 
Prediction:
 [[144.47488]
 [193.0499 ]
 [180.15843]
 [198.59952]
 [149.77237]
 [115.7694 ]
 [146.29878]
 [104.7266 ]
 [187.52466]
 [180.30101]]
50 Cost:  93.59574 
Prediction:
 [[144.50687 ]
 [193.02489 ]
 [180.1671  ]
 [198.60143 ]
 [149.7437  ]
 [115.72924 ]
 [146.30687 ]
 [104.743744]
 [187